# Analyzing Single Neuron Activity

In this tutorial we will examine single-neuron data collected from human patients.

This tutorial was originally developed by 
[Salman Qasim](https://seqasim.wixsite.com/research),
and has been updated by
[Tom Donoghue](https://tomdonoghue.github.io/). 

### Requirements

As well as standard scientific Python packages, this tutorial requires 
[pynwb](https://github.com/NeurodataWithoutBorders/pynwb).

In [ ]:
# Imports - standard scientific Python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.signal import fftconvolve
from scipy.stats import ttest_rel

# Imports - single-unit related 
from pynwb import NWBHDF5IO

In [ ]:
# Import seaborn for plot aesthetics
import seaborn as sns
sns.set_context('talk')

## Overview

The predominant way in which individual neurons communicate with each other is through **action potentials**. These are rapid depolarizations across the neuronal cell membrane that culminate in a neuron releasing neurotransmitters that can do all sorts of things, like excite or inhibit action potentials in other neurons. Because action potentials are characterized by a large and fast change in membrane potential they can be recorded by microwires placed near the neuronal membrane, allowing for direct measurements of individual neuron activity.

Quite differently from local field potentials, at any given moment in time a neuron is either spiking or not. That is, we can think of action potentials as discrete events, and a cell's activity over time as a series of discrete events - or a series of 'spike times'. From this, we can then compute a rate, or the number of spiking events across each unit time. 

When we look at neuronal spiking during a behavioral task, we are often looking for increases in the rate of spiking related to behavioral variables. This is thought to imply a relationship between the stimulus and the neuron's spiking, and is called **rate coding**. Here, we are going to cover some of the basics of analyzing single neuron responses in terms of rate coding.

Note that in our examples here, we will be looking at data that has already been spike sorted, meaning we have isolated markers of neuron action potentials.

In [ ]:
## HELPER FUNCTIONS
# Here, we will define a couple helper functions that we will use throughout the notebook.

def compute_spike_rate(spikes):
    """Estimate spike rate from a vector of spike times, in seconds."""
    
    return len(spikes) / (spikes[-1] - spikes[0])

def get_spike_time_range(spikes, tmin, tmax):
    """Extract spike times for a particular time range."""
    
    return spikes[np.squeeze(np.logical_and([spikes > tmin], [spikes < tmax]))]

## First Dataset: Object Presentation

The first dataset we will use is a an openly available dataset from human patients performing a recognition memory, provided by the 
[Rutishauser Lab](https://www.cedars-sinai.edu/research/labs/rutishauser.html). 

In this task, subjects are presented with pictures of objects, that they are later asked to recall. For our purposes, we will focus on the object presentation, and investigate whether we can find any neurons that seem to relate to visual stimulus presentation, especially of particular object types. 

For convenience, an example subject is included in this repository. The full dataset is available on 
[OSF repository](https://osf.io/cd6qp/), and described in this 
[paper](https://doi.org/10.1038/s41597-020-0415-9).

### Load NWB File

The data that we are loading are in the [NWB](https://www.nwb.org/) format.

In this tutorial, we won't go into much details on NWB files. If you are familiar with HDF5 files, NWB files are actually HDF5 files 'under the hood', with a specific schema for neural data. You can think of them a bit like a Python dictionary, optimized to store heterogeneous and potentially large data. The benefit of these files is that all the data from the experiment 

For more information on NWB files, see the 
[documentation](https://www.nwb.org/)
and/or these
[NWB examples](https://github.com/TomDonoghue/NWBExamples).

In [ ]:
# Load datafile
file_name = 'object_data.nwb'
io = NWBHDF5IO('data/' + file_name, 'r')
nwbfile = io.read()

### Check Units

In [ ]:
# Check how many units in the current file
n_units = len(nwbfile.units)
print(n_units)

In [ ]:
# Set index for the unit of interest
s_ind = 0

In [ ]:
# Extract the waveform for the unit of interest
waveform = nwbfile.units['waveform_mean_encoding'][s_ind, :]

In [ ]:
# Plot the waveform of the unit of interest
plt.plot(waveform)

In [ ]:
# Get spikes from a neuron of interest
spikes = nwbfile.units.get_unit_spike_times(s_ind)

In [ ]:
# Plot a raster of a series of spikes
plt.eventplot(spikes[0:50])
plt.axis('off');

### Interim Summary

So far, we have loaded the data file and accessed some unit data for an example unit. 

Before proceeding, make sure you can access different neuron's, and different time ranges, etc. 

### Event Information

Next, in order to analyze the data with respect to task dynamics, we need to access the event data. 

In NWB, event information is stored as `intervals` type data (markers that denote intervals of interest in the dataset), within which the `trials` data stores structure information about the trial events. 

In [ ]:
# Check the encoding of event information in the NWB file
nwbfile.intervals['trials']

In [ ]:
# Access & check the behavioural information as a dataframe
behav = nwbfile.trials.to_dataframe()
behav.head()

In the dataframe above, we can see various events of interest for the task. 

For our purposes, we are going to focus on visual presentation of stimuli, and will seek to analyze these by stimulus category.

The most relevant events for us are therefore:
- `stim_on_time`, the time in the trial when the stimulus is presented
- `stim_off_time`, the time in the trial when the stimulus is removed
- `category_name`, the object category of the presented stimulus

In [ ]:
# Check the available image conditions
set(nwbfile.intervals['trials'].category_name.data[:])

In [ ]:
# Get the stimulus presentation times
stim_ons = nwbfile.intervals['trials']['stim_on_time'][:]
stim_offs = nwbfile.intervals['trials']['stim_off_time'][:]

In [ ]:
# Get the stimulus onset times for a stimulus category of interest
cond = 'phones'
cond_stim_times = behav[behav.category_name == cond].stim_on_time.values

In [ ]:
# Check the stimulus onset times for our selected object category
cond_stim_times

### Task Related Activity

So far, we have explored the data, accessing the neural and event data. 

Next up, we can use our stimulus presentation times as events of interest, and examine spiking around these times, and try to examine if each neurons spiking is responsive to the presented stimuli.

In [ ]:
def get_trial_spikes(spikes, trial_times, window=1):
    """Extract spike times around event times of interest."""
    
    trial_spikes = []
    for trial_time in trial_times:
        temp = get_spike_time_range(spikes, trial_time - window, trial_time + window)
        trial_spikes.append(temp - trial_time)
        
    return trial_spikes

In [ ]:
# Extract spikes for a unit of interest
s_ind = 0
spikes = nwbfile.units.get_unit_spike_times(s_ind)

In [ ]:
# Collect spikes by trial
trial_spikes = get_trial_spikes(spikes, cond_stim_times)

In [ ]:
# Plot the event-related raster plots
plt.eventplot(trial_spikes);
plt.vlines(0, -1, len(trial_spikes) + 1, color='green', alpha=0.75);
plt.xlim([-1, 1])
plt.axis('off');

#### Statistical Tests

In the above, we have organized our data in order to visualize if there appear to be object related changes in neuron activity. 

In some cases, it might _look_ like there is a change in neuron activity, relating to the stimulus. In other cases, it might look like there is no change, and/or be unclear. In order to more systematically and quantitatively examine this question, we need to do some kind of statistics.

How to statistically test neuron firing is a big topic in single-unit analyses. Conceptually, we want to test whether there is a significant change in firing rate, conditioned on some event of interest. 

Here, we will first start with a simple and conceptually consistent (though non-ideal) test for stimulus related firing - testing for a significant change in firing: a paired t-test on pre & post firing.

In [ ]:
# Reconstruct number of pre & post stim neurons
n_pre, n_post = [], []
for trial in trial_spikes:
    n_pre.append(sum(trial < 0))
    n_post.append(sum(trial > 0))

In [ ]:
# Check for a significant change in firing
ttest_rel(n_pre, n_post)

### Interim Summary

In the above, we have explored a simple approach to examine if single-unit activity systematically relates to visual stimuli.

If we explore the neurons and object categories, you should be able to find some neurons that seem to relate to object category! A neuron with clear encoding will show a visible change in firing, consistent across trials, relative to the stimulus onset time.

We also used a simple statistical test to examine these results. This can help guide our analyses, however we should be careful that t-tests have assumptions about the data, that we didn't test! In fact, there are reasons to believe that our data probably don't follow the assumptions of a t-test. What this means in practice is that our test, and in particular the computed p-value, might not be appropriate. To more rigorously test these associations, we would want to use surrogate statistics.

### Object Dataset: Possible Extensions

So far we explored a simple way to examine if there might be object related activity in the dataset, with simple visualizations and statistical approaches. 

There are many possible way to extend this analysis, that you may want to explore, including:
- Properly examining significance of effects, for example with a permutation procedure
- Generalizing the analysis across multiple neurons
- Correcting for multiple comparisons for the number of neurons
- Exploring further distinctions with the task, for example comparing the `learn` and `recog` phases

## Second Dataset: Spatial Navigation

We will primarily be analyzing data from the Train task, with which we will go over some simple single neuron analyses with respect to spatial position and memory. For reference for this task, see the 
[associated paper](https://www.nature.com/articles/s41593-019-0523-z).

In this task, subjects move down a linear track while encoding and retrieving the locations of objects along the track. In this dataset, we will explore ploting neuronal spiking as a function of time and spatial position, and apply statistical methods for assessing significant changes in both domains. We will also examine measures the influence of memory cues and memory performance on spiking activity. 

The session file we will load is an example session from the train task, also organized into NWB format. This means that 

In [ ]:
# Load data file
file_name = 'spatial_data.nwb'
io = NWBHDF5IO('data/' + file_name, 'r')
nwbfile = io.read()

In [ ]:
# Check how many units in the current file
n_units = len(nwbfile.units)
print(n_units)

In [ ]:
# Set index to access a unit of interest
s_ind = 0

In [ ]:
# Extract spikes for the unit of interest
spikes = nwbfile.units.get_unit_spike_times(s_ind)

In [ ]:
# Check an example selection of spike times
spikes[0:10]

### Descriptive Explorations

In this dataset, let's start by exploring some descriptive measures of spiking activity.

Note that as we can see from the spike times above, the spike times in this file are stored in milliseconds.

#### Firing Rate

An initial, simple measure of unit activity is it's firing rate, which we can compute as the amount of spiking over some unit time.

In [ ]:
# Compute the firing rate of the neuron
#   Note that here we multiple by 1000 to get spikes per second
fr = compute_spike_rate(spikes) * 1000

In [ ]:
# Check the firing rate of the unit of interest
print('The firing rate is: {:2.2f}'.format(fr))

In [ ]:
# Compute the firing rate for all neurons
frs = [compute_spike_rate(nwbfile.units.get_unit_spike_times(ind)) * 1000 \
    for ind in range(n_units)]

In [ ]:
# Plot the firing rate for all neurons
labels = ['U' + str(ind) for ind in range(len(frs))]
_, ax = plt.subplots(figsize=(20, 5))
ax.bar(labels, frs)

This tells you how active, in general, each neuron was during the recording session. 

Checking firing rates can also be used as quality code / pre-selection, since we might want to select neurons with a sufficiently high firing rate, and/or examine if any neurons have suspiciously high firing rates.

#### Inter-Spike Interval

Another way to assess spiking activity is to look at the time interval between each spike (aka the **interspike interval**). 

By computing the inter-spike interval between all spikes, we can examine the distribution of interspike intervals.
This gives us some information about the patterns of unit firing.

In [ ]:
# Compute ISI for a single neuron
isi = np.diff(np.array(spikes))

In [ ]:
# Plot the inter-spike intervals
sns.distplot(isi)
plt.xlabel('ISI (ms)')
plt.ylabel('density')

#### Coefficient of Variation

From the ISI distribution, you can compute the **coefficient of variation**. 

Note that the CV does not capture potential variability on longer time scales especially if there's drift in the neuron's mean firing rate over time. 

As an aside, another useful thing that ISI distribution can tell you is if the neuron you are looking at is a **bursty** neuron. By that, I mean a neuron that tends to fire a lot of action potentials in short bursts, rather than as isolated single spikes. As you can imagine, bursty neurons tend to violate Poisson assumptions and have ISI distributions that look a little more bimodal, with lots of spikes close together, and lots of bursts far apart.

In [ ]:
# Compute coefficient of variation
cv = np.std(isi) / np.mean(isi)

In [ ]:
# Check the computed CV
print(cv)

### Data Representations

Before we continue to examine the neural firing with respect to the task, let's consider and explore some different ways to represent the data.

#### Spike Times

So far, we have been using **spike times**, literally a list of the times at which a unit fired.

#### Spike Trains

Another way we can represent the data is a **spike train**, which is binary representation of 0's and 1's, in which each value represents whether the unit is spikig (1) or not (0). 

#### Continuous Firing Rates

Note that if we are interested in rates, then we might be more interested in the overall rate of activity, than exact times at which spikes occur. One main reason to use spike trains is in order to compute continuous firing rates. By sweeping across windows in the soike train, and computing the firing rate within each time bin, we can compute a continuous measure of spike rate over time.

### Computing Spike Trains

Let's now compute and look at some spike trains. 

In [ ]:
# Settings
bin_width_st = 1   # this means our time resolution is 1 ms
st_sr = 1000       # This is our sampling rate 

Note that one thing we can also do now is to check and extract a time range specific to the task. Note that in the data collection, unit recordings might start before the task, and/or extend beyond the tak time. For the next step, let's define a spike train, selecting the spikes that happened during the task.

In [ ]:
# Extract start & stop time of the task from the event log
start = nwbfile.intervals['trials'].start_time[0]
end = nwbfile.intervals['trials'].stop_time[-1]

In [ ]:
# Extract spikes from during the task time
#   First we find indices to keep, and then re-align time to 0
keep_inds = np.where(np.logical_and(spikes>=start, spikes<=end))[0].astype(int)
task_spikes = spikes[keep_inds] - start

In [ ]:
# Spike times are in ms, but are sampled at 30 KHz 
#   This means we have to round them to ms resolution
rounded_spike_times = np.round(task_spikes).astype(int)

# Generate a spike train - a binary vector of 0's and 1's that is indexed by the spike times. 
n_vals = int(((np.ceil(end) - np.floor(start)) / bin_width_st) + 1)
spiketrain = np.zeros(n_vals)

# Add the spikes - note we have to subtract 1 for pythonic indexing
spiketrain[rounded_spike_times - 1] = 1 

In [ ]:
# Plot the spike train
plt.plot(spiketrain[0:1000])

#### Fano Factor

A measure of spike time variability taking this long-term variability into account is the Fano factor, the ratio of the mean spike count and the variance the spike count within a time window.

In [ ]:
# Now we can compute the Fano Factor, which requires the spike train , during the task
fano_factor = np.var(spiketrain) / np.mean(spiketrain)
print('This cell has a Fano factor of {}'.format(fano_factor))

### Computing Continous Firing Rates

One of the key reasons to define a spike train is so that we can smooth it to get a continuous **estimate** of spiking. 

We can do this by convolving the spike train with a Gaussian kernel.

In [ ]:
# Continuous firing rate parameters
bin_width_gauss = 10                        # bin width, in ms
gauss_sr = int(1000 / bin_width_gauss)      # gaussian sampling rate
smoothing_width = 75                        # gaussian kernel, in ms 

In [ ]:
# Extract the time stamps of the position data
#   We can use this time vector to organize time bins, to match position data
times = nwbfile.acquisition['position']['position'].timestamps[:]

In [ ]:
# Make the time bins 
times_offset = times - times[0]
binned_time = np.arange(times_offset[0], times_offset[-1] + np.diff(times_offset)[0], bin_width_gauss)

# Map the spike times to bins
spkt = np.zeros(binned_time.shape)
map_to_bins = np.digitize(task_spikes, binned_time)
for i in map_to_bins:
    if i > 0:
        spkt[i - 1] += 1

In [ ]:
# Define a window to filter in 
filt_window = np.arange(-1000, 1000, bin_width_gauss)

# factor to convert the convolved firing rate into Hz (spikes/second)
conv_rate_gaussian_fr = 1 / (bin_width_gauss / 1000) 

# Define the gaussian kernel
gaussian_kernel = 1. / np.sqrt(2 * np.pi * smoothing_width ** 2) * np.exp(
    -filt_window ** 2. / (2 * smoothing_width ** 2))

# Normalize the kernel so that the area sums to 1
gaussian_kernel =  gaussian_kernel / gaussian_kernel.sum()

# Do the smoothing!
spkt_conv = conv_rate_gaussian_fr * fftconvolve(spkt, gaussian_kernel, 'same') 

In [ ]:
# Plot s of data
f, (gauss, train) = plt.subplots(2, 1, figsize=[18,3])
time_win = 100
gauss.plot(spkt_conv[0:(time_win * gauss_sr)])
train.plot(spiketrain[0:(time_win * st_sr)], linewidth=0.2)

## Task Analysis

Now let's look at how neural spiking changes in relation to task events.

In general, the most useful tool for visualizing stimulus-related changes in the firing rate is the **peri-event rasters and histograms**. 

First, we have to do some data wrangling. The events here are not organized by trial/event, so we need to do that, then use our baselined spikes and the timesoffset field to parse the spiking the same way. 
....

In [ ]:
# Access & check the behavioural information as a dataframe
behav = nwbfile.trials.to_dataframe()
behav.head()

In [ ]:
# Extract the time for the responses per trial
all_response_times = nwbfile.intervals['trials'].response_time.data[:]

In [ ]:
# Define a time window of interest for around events
time_window = [-500, 500]

In [ ]:
# Extract the spikes surrounding each stimulus 
spike_times_per_event = [] 
for response in all_response_times:
    keep_event_inds = np.where(np.logical_and(task_spikes >= response + time_window[0], 
                                              task_spikes <= response + time_window[1]))[0].astype(int)
    spike_times_per_event.append(task_spikes[keep_event_inds] - response)

In [ ]:
# Plot the raster
f, (raster, spike_hist) = plt.subplots(2, 1, figsize=[6,6])
trial = 1
for row in spike_times_per_event: 
    raster.vlines(row, trial, trial+1)
    trial += 1
raster.vlines(0, 0, trial)

# Compute the histogram in 50 ms bins
bin_width_ms = 50 
n_bins = int(1000 / bin_width_ms)
H, b = np.histogram(np.hstack(spike_times_per_event), n_bins)
rate_factor = (bin_width_ms * len(nwbfile.intervals['trials']) / 1000)

sns.distplot(np.hstack(spike_times_per_event), bins=n_bins, kde=False, ax=spike_hist)
# Scale the y-axis so we are plotting firing rate in our bins, not just counts of spikes 
y_vals = spike_hist.get_yticks()
spike_hist.set_yticklabels(['{:3.1f}'.format(x/rate_factor) for x in y_vals])
spike_hist.vlines(0, 0, y_vals.max())

### Analysis Notes

Similar to our analysis of the previous dataset, we now have the question of how to do statistics for this kind of **peri-stimulus time histogram**? As we explored before, one option is to compute the mean firing rate pre & post and do a paired t-test.

As we mentioned before, there are other ways to examine this question, including, for example, if we wanted to find particular time points in which there might be a change in firing rate. For example, to identify specific times of increased firing, we could generate surrogate PSTHs (i.e. 500 null observations per time bin), and compute a p-value for every bin to examine if any time bins in the non-shuffled real data exceed the empirical (shuffled) null. From there, one can do multiple comparisons correction across time bins to examine if there are any time bins that are significantly higher/lower than surrogates. 

We won't try and do this right now, but this is left as a potential extension of the analysis, and something we will follow up on in the extended work section at the end. Note that the the permutation statistics mentioned in the extended section will be very similar to what you might do for as PSTH. 

## Spatial Encoding Analysis

Now, let's examine neural activity as a function of position.

Up until this point, we have been binning spike counts by **time**. Now it's time to bin spike counts by **position**.

The easiest way to do this is to utilize pandas dataframe functionality, particularly the `cut` function to cut data into bins and the `groupby` function to apply a function to these split data. 

In [ ]:
# Extract the position data from the file
position = nwbfile.acquisition['position']['position']
times = position.timestamps
pos = position.data

In [ ]:
# Examine some position traces
plt.plot(pos[0:1000])

In [ ]:
# Cut the environment into spatial bins, also collecting the bin edges
n_spatial_bins = 20
spatial_bins, bin_edges = pd.cut(pos[:], bins=n_spatial_bins, retbins=True,
                                 include_lowest=True, labels=np.arange(n_spatial_bins))

In [ ]:
# Examine the bin assignment of each position
spatial_bins

### Create a dataframe

What we want to do next is examine if our single-neuron activity relates to task elements of interest - in this case spatial location and object presence. Note that now we have multiple behavioural features of interest, and we are going to have to take a different analysis strategy to examine how neural firing relates to multiple features of interest. 

To do this, we need a representation of the data that organized neural data, with measures of interest including space, objects, and neural activity. This requires some design choices, including, for example:
- How do we encode our variable of interest?
    - One could, for example, encode space in continuous or discrete ways
- How do model the firing rate?
    - One could, for example, encode firing rate as discrete firing rates, or a modeled as a Poisson process
    
For our purposes, we are going to discretize space into bins, use firing rates for our neural activity, and organize our data so that for each sampled time bin, we have spatial bin, trial object, and neural firing rate. 

To get started, for convenience, we can load a dataframe that has already been prepared this representation of the data. This dataframe includes the organized data for the 1st neuron in the dataset. For further analyses, and/or for other neurons, you can recreate these dataframes for each cell. 

In [ ]:
# Load a dataframe of organized data for the current analysis
data_df = pd.read_csv('data/spatial_dataframe.csv')

In [ ]:
# Check out the loaded dataframe
data_df

In [ ]:
# Compute the actual spatially binned firing rate by averaging the firing rate over each bin 
spatially_binned_fr = data_df.groupby(by='spatial_bin').mean()

In [ ]:
# Plot the firing across spatial bins
#   Note that seaborn does the groupby().mean() itself 
sns.lineplot(x='spatial_bin', y='fr', data=data_df, ci=68) 

In the above, we can see how the spatial firing relates to spatial position across the linear track.

In [ ]:
# Compute the firing rate for each different object cued for memory 
sns.barplot(x='object_ID', y='fr', data=data_df)

In the above, we can see how the the average firing of the cell relates to which object is on the track.

In the above plot, firing for each object is collapsed across trials (across both space and time). Next, we can examine the firing for the cell, split by object, across space. 

In [ ]:
# Does spatial firing rate differ as a function of which object is cued for retrieval? 
diff_objects = data_df.object_ID.unique()
fig, axes = plt.subplots(2, 2, figsize=[12, 8])

for ind, obj in enumerate(axes.flatten()):
    sns.lineplot(x='spatial_bin', y='fr', data=data_df[data_df.object_ID==diff_objects[ind]], ci=68, ax=obj)
    obj.vlines(data_df[data_df.object_ID==diff_objects[ind]].object_bin, 0, obj.get_ylim()[-1])
    obj.set_title(diff_objects[ind])
    obj.set_ylim([0, 22])

fig.tight_layout()

### Statistical Analysis

Next, we need to find a way to do examine these patterns of firing statistically. To do so, with multiple behavioural features of interest (multiple locations & multiple objects), we are going to try and fit a model to see if & to what extent we can explain variance of the neural firing in terms of our events of interest.

Though by no means the only way to do it, here we will take a fairly standard approach: using an ANOVA to investigate if binned spatial position and/or the trial object relate to cell firing. 

The goal of the model we want to fit is to statistically determine if this neuron shows significant spatial tuning, significant object tuning, or an interaction of the two. Let's discuss each of these possibilites:

1) Significant spatial tuning: A neuron significantly increases it's firing in a particular location. We would call this a place cell. 

2) Significant object tuning: A neuron significantly increases it's firing when a particular object is cued for memory retrieval. 

3) Significant spatial x object: A neuron significantly increases it's firing in a particular location **as a function of the object cued for memory**. Put another way, the memory a person is cued to retrieve is affecting the spatial tuning of the neuron.

To assess these possibilities in an individual neuron, we are going to do a simple 2-way ANOVA. 

In [ ]:
# Define the formula of interest that we want to test
formula = 'fr~C(spatial_bin) + C(object_ID) + C(spatial_bin)*C(object_ID)'

In [ ]:
# Fit the model
model = ols(formula, data_df).fit()
aov_table = anova_lm(model, typ=2)

# Grab variables of interest
F_int = aov_table['F']['C(spatial_bin):C(object_ID)']
F_pos = aov_table['F']['C(spatial_bin)']
F_obj = aov_table['F']['C(object_ID)']

In [ ]:
# Check the model fit results
aov_table 

Note that in the above, this neuron appears to have significant encoding of some features of interest!

However, we must keep in mind that our data might violate normality assumptions for computing the significance of the test-statistic (F)!

In order to more robustly evaluate the statistics, we should compute our own null distributions from surrogate data to assess significance.

## Follow up Analyses

So far, we have examined, for an single-cell, a first-pass analysis of if this cell's activity seems to relate to place and or object encoding. There is of course, much more we could explore here, and much we could generalize to analyze across multiple cells, sessions, etc, and to improve the statistical analyses. 

For this extended analyses, the goal is to further analyze human single neuron activity during a virtual-reality spatial memory task.

#### 1) Explore PSTHs for other events

First, try and write a general function to plot the raster and histogram at a user defined time window and bin length (for the histogram). Use it to plot the rasters and histograms for CueOn, CueOff, FeedbackOn and FeedbackOff. This function should use baselined spike times and baselined event times/trial as input. Plot the output for one neuron.

#### 2) Explore smoothed firing rates for other events

Write a similar function using the smoothed spiking activity instead of the PSTH. Keep in mind that the we computed the smoothed spiking activity at 100 Hz. Be sure to allow users to input width of smoothing kernel. You may want to write a separate function to smooth the firing rate, and call that within your function to plot the raster + smoothed firing rate. Plot the output for one neuron, with 3 different smoothing widths for your kernel. 

#### 3) Examine neural correlates of performance

Calculate the mean firing rate and the error on every trial, and plot firing rate as a function of error. 

Hint: to do this, you are going to have to use the event information in the data file. Based on the object location, and the response location, you can calculate the behavioural error error per trial. Using the trial start and end times, you can also extract firing for these times of interest, and examine firing in trials as a function of error. 

BONUS: Is there a statistical relationship between firing rate and memory performance? 

#### 4) Generalize the ANOVA analysis

Finally, see if you can generalize the analysis ANOVA analysis we did, both to generalize it across all cells, and also to add surrogate analysis to have more robust statistics.

To do so, first you will need to write a helper function that organizes the data from the raw file into the kind of dataframe we used to fit the model. Following that dataframe, try and write a function to create this dataframe for each neuron. 

Then try and do some shuffling to determine if the F-statistic from the spatial ANOVA is significant. 

Hint: use np.roll() to circularly shift data in a pandas dataframe: 
`df.reindex(index=np.roll(df.index, shift))`.

If you see any neurons with a significant interaction in the ANOVA, make a plot of the spatial firing rate split by object. If you see any neurons that do not have a significant interaction but do have a significant main effect of location, make a plot of the spatial firing rate over all trials. 
